mysql과 python 연동
1. 라이브러리 설치
2. sql 서버의 정보를 등록
3. sql 서버에 접속
4. sql 쿼리문을 이용하여 데이터를 CRUD 작업

In [1]:
# pymysql 설치
!pip install pymysql

     ---------------------------------------- 43.7/43.7 kB 2.2 MB/s eta 0:00:00


In [2]:
import pymysql

In [6]:
# sql 서버의 정보를 입력
_db = pymysql.connect(
    user = 'root',
    password = '0630',
    host = 'localhost', 
    db = 'ubion7', #데이터베이스명
    port = 3306
)

In [31]:
# cursor 변수 생성
# sql 쿼리문을 실행 시키기 위한 필수 변수
# Dictcursor는 sql 쿼리문이 실행되고 나온 데이터를 딕셔너리 형태로 받는다.
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [14]:
sql = """
    select
    *
    from
    user_info
"""

# """ 하면 한 줄 대신 위에 이렇게 쓸 수 있음 """
cursor.execute(sql)

2

In [15]:
result = cursor.fetchall()
result

[{'user_id': 'test',
  'user_pass': '1111',
  'user_name': 'test',
  'user_phone': '01012345678',
  'user_gender': 1},
 {'user_id': 'test2',
  'user_pass': '1234',
  'user_name': 'test2',
  'user_phone': '01011112222',
  'user_gender': 1}]

In [11]:
import pandas as pd

In [16]:
pd.DataFrame(result)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1111,test,01012345678,1
1,test2,1234,test2,01011112222,1


In [17]:
sql = """
    select
    *
    from
    example
"""

cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df

,Name,Gender,Phone
0,A,male,010-1234-5678
1,B,female,010-1234-5679
2,C,male,010-1234-5680
3,D,female,010-1234-5681
4,E,male,010-1234-5682
5,F,female,010-1234-5683
6,G,male,010-1234-5684
7,H,female,010-1234-5685
8,I,male,010-1234-5686
9,J,female,010-1234-5687


In [20]:
sql = """
    insert into
    user_info
    values
    ("test3", "1234", "test3", "01011118888" , 1)
"""

cursor.execute(sql)

1

In [21]:
_db.commit() #이거까지 해야 반영됨

### sql 함수화
1. 첫번째 함수
    - 매개변수 1개(sql)
    - sql 쿼리문을 실행하고 결과를 데이터프레임으로 변경하여 리턴
2. 두번째 함수
    - 매개변수 1개(sql 쿼리문)
    - sql 쿼리문을 실행하고 데이터베이스를 변경
3. 세번째 함수
    - 1번 함수와 2번 함수를 하나의 함수에서 실행이 될 수 있도록 생성

In [38]:
def func_1(sql):
    cursor.execute(sql)

    result = cursor.fetchall()

    func_1 = pd.DataFrame(result)
    return func_1

In [39]:
sql = """
select
*
from
user_info
"""

func_1(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1111,test,01012345678,1
1,test2,1234,test2,01011112222,1
2,test3,1234,test3,01011118888,1
3,test4,2222,test4,01022229999,1


In [40]:
def func_2(sql):
    cursor.execute(sql)
    _db.commit()

In [41]:
sql = """
    insert into
    user_info
    values
    ("test5", "5555", "test5", "01077779999" , 1)
"""

func_2(sql)

In [44]:
def func_2(sql,values):
    cursor.execute(sql, values)
    _db.commit()

In [45]:
sql = """
    insert into
    user_info
    values
    (%s, %s, %s, %s, %s)
"""

values= ["test7", "7777", "test7", "01078879999" , 1]

func_2(sql,values)

In [48]:
sql = """
    select
    *
    from
    user_info
"""

sql

'\n    select\n    *\n    from\n    user_info\n'

In [49]:
sql.replace('\n', '')

'    select    *    from    user_info'

In [50]:
sql.replace('\n', '').strip()

'select    *    from    user_info'

In [47]:
sql = """
    select
    *
    from
    user_info
"""
print(sql.replace('\n', '').strip().startswith('select'))

True


In [53]:
def func_3(sql,values=[]):
    # if (sql.find('select') != -1) and (sql.find('select') < 10):
    if (sql.replace('\n', '').strip().startswith('select')):
        cursor.execute(sql,values)
        result = cursor.fetchall()
        result = pd.DataFrame(result)
    else:
        cursor.execute(sql,values)
        _db.commit()
        result = "Query OK"
    return result

In [54]:
sql = """
    select
    *
    from
    user_info
"""

func_3(sql)

In [56]:
sql = """
    insert into
    user_info
    values
    (%s, %s, %s, %s, %s)
"""

values= ["test8", "8888", "test8", "01012345678" , 1]

func_3(sql,values)

'Query OK'

In [57]:
sql = """
    delete
    from
    user_info
    where
    user_id = %s
"""

values= ["test8"]

func_3(sql,values)

'Query OK'